In [57]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [58]:
x_train_a = pd.read_csv('cleaned_data/A/x_train_a.csv')
x_train_b = pd.read_csv('cleaned_data/B/x_train_b.csv')
x_train_c = pd.read_csv('cleaned_data/C/x_train_c.csv')

x_test_a = pd.read_csv('cleaned_data/A/x_test_a.csv')
x_test_b = pd.read_csv('cleaned_data/B/x_test_b.csv')
x_test_c = pd.read_csv('cleaned_data/C/x_test_c.csv')

train_a = pd.read_csv('cleaned_data/A/train_a.csv')
train_b = pd.read_csv('cleaned_data/B/train_b.csv')
train_c = pd.read_csv('cleaned_data/C/train_c.csv')

In [59]:
train_a['time'] = pd.to_datetime(train_a['time'])
train_b['time'] = pd.to_datetime(train_b['time'])
train_c['time'] = pd.to_datetime(train_c['time'])

In [60]:
x_test_a = x_test_a.drop(columns = ['date_forecast'])
x_test_b = x_test_b.drop(columns = ['date_forecast'])
x_test_c = x_test_c.drop(columns = ['date_forecast'])


In [61]:
x_test_a.dtypes

absolute_humidity_2m:gm3          float64
air_density_2m:kgm3               float64
ceiling_height_agl:m              float64
clear_sky_energy_1h:J             float64
clear_sky_rad:W                   float64
cloud_base_agl:m                  float64
dew_or_rime:idx                   float64
dew_point_2m:K                    float64
diffuse_rad:W                     float64
diffuse_rad_1h:J                  float64
direct_rad:W                      float64
direct_rad_1h:J                   float64
effective_cloud_cover:p           float64
elevation:m                       float64
fresh_snow_12h:cm                 float64
fresh_snow_1h:cm                  float64
fresh_snow_24h:cm                 float64
fresh_snow_3h:cm                  float64
fresh_snow_6h:cm                  float64
is_day:idx                        float64
is_in_shadow:idx                  float64
msl_pressure:hPa                  float64
precip_5min:mm                    float64
precip_type_5min:idx              

In [62]:
x_train_a.dtypes

date_forecast                      object
absolute_humidity_2m:gm3          float64
air_density_2m:kgm3               float64
ceiling_height_agl:m              float64
clear_sky_energy_1h:J             float64
clear_sky_rad:W                   float64
cloud_base_agl:m                  float64
dew_or_rime:idx                   float64
dew_point_2m:K                    float64
diffuse_rad:W                     float64
diffuse_rad_1h:J                  float64
direct_rad:W                      float64
direct_rad_1h:J                   float64
effective_cloud_cover:p           float64
elevation:m                       float64
fresh_snow_12h:cm                 float64
fresh_snow_1h:cm                  float64
fresh_snow_24h:cm                 float64
fresh_snow_3h:cm                  float64
fresh_snow_6h:cm                  float64
is_day:idx                        float64
is_in_shadow:idx                  float64
msl_pressure:hPa                  float64
precip_5min:mm                    

# Analysis of Target variable  - Looking at PV_measurement
1. Handle constant measurments over longer periods of time. Likely caused by sensor malfunction, data logging issues, or other external factors.
    - Handeled by removing all constant values lasting more than 24 hours 
2. Add cyclical features 
2. Handle longer periods of missing data:
    - Remove (currently tested)
    - Interpolate 
    - Copy from previous year
    - Copy solar production at missing time from another location

### 1. Handle constant PV measurements 

In [63]:
# Time-Series plot of PV_measurement 

def solar_prod_plot(y_train, resolution='year', chunks=5):
    df = y_train.copy()
    
    # Determine the plotting resolution based on the 'resolution' argument
    # Chunks = number of year/months/days in each plot
    if resolution == 'year':
        unique_values = df['time'].dt.year.unique()
        label = 'Year'
    elif resolution == 'month':
        df['year_month'] = df['time'].dt.to_period('M')
        unique_values = df['year_month'].unique()
        label = 'Month'
    elif resolution == 'week':
        df['year_week'] = df['time'].dt.to_period('W')
        unique_values = df['year_week'].unique()
        label = 'Week'
    elif resolution == 'day':
        df['date'] = df['time'].dt.date
        unique_values = df['date'].unique()
        label = 'Day'
    else:
        raise ValueError("Invalid resolution. Choose from 'year', 'month', 'week', or 'day'.")
    
    # Loop over the unique values in chunks
    for i in range(0, len(unique_values), chunks):
        subset_values = unique_values[i:i+chunks]
        
        if resolution == 'year':
            subset_df = df[df['time'].dt.year.isin(subset_values)]
        elif resolution == 'month':
            subset_df = df[df['year_month'].isin(subset_values)]
        elif resolution == 'week':
            subset_df = df[df['year_week'].isin(subset_values)]
        elif resolution == 'day':
            subset_df = df[df['date'].isin(subset_values)]
        
        plt.figure(figsize=(15, 6))
        plt.plot(subset_df['time'], subset_df['pv_measurement'])

        title = f"Solar Power Production for {label}: {subset_values[0]}"
        if len(subset_values) > 1:
            title += f" to {subset_values[-1]}"

        plt.title(title)
        plt.xlabel("Time")
        plt.ylabel("PV Measurement")
        plt.show()

def remove_constant_intervals(y_train, low_thresh, upp_thresh):
    """
    Identify and remove intervals of constant PV readings that exceed a specified duration. 
    Constant readings may indicate sensor malfunctions or data logging issues.
    
    Parameters:
    ----------
    y_train : pd.DataFrame
        Dataframe containing the time-series data of solar power production.
    threshold : int
        The minimum duration required for an interval to be considered for removal.
        
    Returns:
    -------
    pd.DataFrame
        The input dataframe with intervals of constant readings (exceeding the duration threshold) removed.
    """
    df = y_train.copy()
    
    # Calculate the difference in production values
    df['diff'] = df['pv_measurement'].diff()

    # Identify where the difference is zero
    df['zero_diff'] = df['diff'].abs() < 1e-5

    # Identify groups of consecutive zero differences
    df['group'] = (df['zero_diff'] != df['zero_diff'].shift()).cumsum()

    # Filter out only the groups with consecutive zero differences
    constant_intervals = df[df['zero_diff']].groupby('group').agg(start=('time', 'min'), 
                                                                  end=('time', 'max'),
                                                                  duration=('time', 'size'))
    
    # Filter intervals based on the threshold
    interval_df_thresh = constant_intervals[(constant_intervals['duration'] > low_thresh) & (constant_intervals['duration'] <upp_thresh)]
    
    # Remove rows from the main dataframe that fall within these intervals
    for _, row in interval_df_thresh.iterrows():
        start_time, end_time = row['start'], row['end']
        df = df[(df['time'] < start_time) | (df['time'] > end_time)]
    
    # Drop the added columns used for calculations
    df.drop(columns=['diff', 'zero_diff', 'group'], inplace=True)
    
    return df, constant_intervals


def get_time_interval(df, start_time = '2020-08-01 00:00:00', end_time = '2021-01-01 00:00:00'):
    # Filter rows based on the time period
    filtered_df = df[(df['time'] >= start_time) & (df['time'] <= end_time)]
    return filtered_df

In [64]:
#Removed all constant values with duration > 24 hours

train_a, const_interval_a = remove_constant_intervals(train_a,24,10**6)

#update X_train_a by removing coresponding rows that have been filtered here
x_train_a, train_a = align_X_y(x_train_a, train_a)

In [65]:
rows_removed_a = np.sum(const_interval_a[const_interval_a['duration']>24]['duration'])
print(f'total number of rows removed {rows_removed_a}')
const_interval_a[const_interval_a['duration']>24]

total number of rows removed 42


,start,end,duration
group,,,
434,2020-01-04 15:00:00,2020-01-06 08:00:00,42


In [66]:
#Remove rows in groups of constant values, where duration of constant measurements is > 1 day (24 hours)
train_b, const_interval_b = remove_constant_intervals(train_b,24,10**6)

#update X_train_a by removing coresponding rows that have been filtered here
x_train_b, train_b = align_X_y(x_train_b, train_b)

In [67]:
rows_removed = np.sum(const_interval_b[const_interval_b['duration']>24]['duration'])
print(f'total number of rows removed {rows_removed}')
const_interval_b[const_interval_b['duration']>24]

total number of rows removed 6865


,start,end,duration
group,,,
32,2019-01-14 15:00:00,2019-01-18 11:00:00,93
36,2019-01-19 13:00:00,2019-01-26 08:00:00,164
40,2019-01-27 11:00:00,2019-01-28 13:00:00,27
74,2019-02-10 16:00:00,2019-02-13 07:00:00,64
160,2019-03-23 18:00:00,2019-03-26 06:00:00,61
302,2019-05-31 08:00:00,2019-06-03 12:00:00,77
606,2019-10-28 14:00:00,2019-10-30 22:00:00,57
674,2019-12-01 13:00:00,2019-12-04 08:00:00,68
682,2019-12-07 14:00:00,2019-12-11 08:00:00,91


In [68]:
#Remove rows in groups of constant values, where duration of constant measurements is > 1 day (24 hours)
train_c, const_interval_c = remove_constant_intervals(train_c,24,10**6)

#update X_train_a by removing coresponding rows that have been filtered here
x_train_c, train_c = align_X_y(x_train_c, train_c)

In [69]:
rows_removed = np.sum(const_interval_c[const_interval_c['duration']>24]['duration'])
print(f'total number of rows removed {rows_removed}')
const_interval_c[const_interval_c['duration']>24]

total number of rows removed 4926


,start,end,duration
group,,,
2,2019-09-04 10:00:00,2019-09-05 12:00:00,27
180,2019-11-11 12:00:00,2019-11-13 08:00:00,45
230,2019-11-28 15:00:00,2019-12-05 09:00:00,163
240,2019-12-07 14:00:00,2019-12-13 09:00:00,140
256,2019-12-16 14:00:00,2019-12-21 09:00:00,116
276,2019-12-25 13:00:00,2019-12-30 09:00:00,117
290,2020-01-02 14:00:00,2020-01-07 09:00:00,116
340,2020-01-23 15:00:00,2020-01-26 08:00:00,66
376,2020-02-05 14:00:00,2020-02-10 07:00:00,114


### Merge x_train and y_train

In [70]:
merged_a = pd.merge(x_train_a, train_a, left_on='date_forecast', right_on='time', how='inner')
merged_b = pd.merge(x_train_b, train_b, left_on='date_forecast', right_on='time', how='inner')
merged_c = pd.merge(x_train_c, train_c, left_on='date_forecast', right_on='time', how='inner')

In [71]:
#Here we are plotting on the modified dataset
def time_series_plot(feature,merged_data):
    fig, ax1 = plt.subplots(figsize=(15, 6))
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Solar Power Production', color='tab:blue')
    ax1.plot(merged_data['time'], merged_data['pv_measurement'], color='tab:blue', label='Solar Power Production')
    ax1.tick_params(axis='y', labelcolor='tab:blue')

    ax2 = ax1.twinx()  
    ax2.set_ylabel(feature, color='tab:red')  
    ax2.plot(merged_data['date_forecast'], merged_data[feature], color='tab:red', label=feature)
    ax2.tick_params(axis='y', labelcolor='tab:red')

    fig.tight_layout()
    plt.title(f'Time Series Plot of Solar Power Production and {feature}')
    plt.show()

### Add avg pv_feature

In [ ]:
import pandas as pd

def add_average_pv_feature(merged_df, test_df,time_group):
    df = merged_df.copy()
    test_df = test_df.copy()
    # Group by year, month, date, and hour and calculate the mean PV measurement
    average_pv = df.groupby(time_group)['pv_measurement'].mean().reset_index()
    average_pv = average_pv.rename(columns={'pv_measurement': 'average_pv_measurement'})

    # Print for debugging


    # Merge the average PV measurements back into the original dataframe
    df = pd.merge(df, average_pv, on=time_group, how='left')
    test_df = pd.merge(test_df, average_pv, on= time_group, how='left')
    
    # Print for debugging
    return df,test_df

"""
time_group_1 = ['month', 'day', 'hour']
time_group_2 = 'week'
time_group_3 = 'month'

merged_a_avg, x_test_a_avg = add_average_pv_feature(merged_a,x_test_a,time_group_1)
merged_b_avg, x_test_b_avg = add_average_pv_feature(merged_b,x_test_b, time_group_1)
merged_c_avg, x_test_c_avg = add_average_pv_feature(merged_c,x_test_c,time_group_1)

merged_a_avg[(merged_a_avg['month']==6) & (merged_a_avg['day']==4) & (merged_a_avg['hour']==16)][['year','month','week','day','hour','pv_measurement','average_pv_measurement']]
"""

### Add lag features

In [ ]:
def add_lag_feature(data, lag_hours, column_name='pv_measurement'):
    """
    Add lag features to the dataset.

    Parameters:
    data (pd.DataFrame): The original dataset.
    lag_hours (int): The number of hours to lag.
    column_name (str): The name of the column to create the lag feature for.

    Returns:
    pd.DataFrame: The dataset with the new lag feature.
    """

    # Create the lag feature
    df = data.copy()
    lag_feature_name = f"{column_name}_lag_{lag_hours}h"
    df[lag_feature_name] = df[column_name].shift(lag_hours)

    return df

"""
laged_a = add_lag_feature(merged_a,24)
laged_b = add_lag_feature(merged_b,24)
laged_c = add_lag_feature(merged_c,24)

x_test_a_laged = x_test_a.copy()
x_test_b_laged = x_test_b.copy()
x_test_c_laged = x_test_c.copy()


# You can add an empty column for the lag feature in your test set:
x_test_a_laged[f'pv_measurement_lag_{1}h'] = None
x_test_b_laged[f'pv_measurement_lag_{1}h'] = None
x_test_c_laged[f'pv_measurement_lag_{1}h'] = None
""

### Handle NaN values

In [17]:
merged_a.isna().sum().sort_values(ascending = False)
merged_b.isna().sum().sort_values(ascending = False)
merged_c.isna().sum().sort_values(ascending = False)

snow_density:kgm3                 21110
ceiling_height_agl:m               4421
cloud_base_agl:m                   1886
pressure_50m:hPa                     24
sun_azimuth:d                        24
prob_rime:p                          24
rain_water:kgm2                      24
relative_humidity_1000hPa:p          24
sfc_pressure:hPa                     24
snow_depth:cm                        24
snow_drift:idx                       24
snow_melt_10min:mm                   24
snow_water:kgm2                      24
sun_elevation:d                      24
pressure_100m:hPa                    24
super_cooled_liquid_water:kgm2       24
t_1000hPa:K                          24
total_cloud_cover:p                  24
visibility:m                         24
wind_speed_10m:ms                    24
wind_speed_u_10m:ms                  24
wind_speed_v_10m:ms                  24
wind_speed_w_1000hPa:ms              24
absolute_humidity_2m:gm3             24
precip_type_5min:idx                 24


In [45]:
def remove_nan(merged_data):
    df = merged_data.copy()
    
    nan_cols = ['absolute_humidity_2m:gm3', 'air_density_2m:kgm3',
       'ceiling_height_agl:m', 'clear_sky_energy_1h:J', 'clear_sky_rad:W',
       'cloud_base_agl:m', 'dew_or_rime:idx', 'dew_point_2m:K',
       'diffuse_rad:W', 'diffuse_rad_1h:J', 'direct_rad:W', 'direct_rad_1h:J',
       'effective_cloud_cover:p', 'elevation:m', 'fresh_snow_12h:cm',
       'fresh_snow_1h:cm', 'fresh_snow_24h:cm', 'fresh_snow_3h:cm',
       'fresh_snow_6h:cm', 'is_day:idx', 'is_in_shadow:idx',
       'msl_pressure:hPa', 'precip_5min:mm', 'precip_type_5min:idx',
       'pressure_100m:hPa', 'pressure_50m:hPa', 'prob_rime:p',
       'rain_water:kgm2', 'relative_humidity_1000hPa:p', 'sfc_pressure:hPa',
       'snow_density:kgm3', 'snow_depth:cm', 'snow_drift:idx',
       'snow_melt_10min:mm', 'snow_water:kgm2', 'sun_azimuth:d',
       'sun_elevation:d', 'super_cooled_liquid_water:kgm2', 't_1000hPa:K',
       'total_cloud_cover:p', 'visibility:m', 'wind_speed_10m:ms',
       'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms', 'wind_speed_w_1000hPa:ms']
    
    nan_rows_mask = merged_data.loc[:,nan_cols].isna().all(axis=1)
    df = df.drop(df[nan_rows_mask].index, inplace=False)
    
    df = df.drop(columns = ['snow_density:kgm3']) #Tried also to remove 'cloud_base_agl:m' and ceiling_height_agl:m 
    return df



In [46]:
merged_a = remove_nan(merged_a)
merged_b = remove_nan(merged_b)
merged_c = remove_nan(merged_c)

x_test_a = remove_nan(x_test_a)
x_test_b = remove_nan(x_test_b)
x_test_c = remove_nan(x_test_c)

### Add Cyclical Features

In [72]:
# Creating cyclical features for hour of the day
def add_cyclic(merged_df):
    train_data = merged_df.copy()
   
    train_data['hour_sin'] = np.sin(2 * np.pi * train_data['hour'] / 24)
    train_data['hour_cos'] = np.cos(2 * np.pi * train_data['hour'] / 24)
    train_data['month_sin'] = np.sin(2 * np.pi * (train_data['month']-1) / 12)
    train_data['month_cos'] = np.cos(2 * np.pi * (train_data['month']-1) / 12)
    
    #train_data.drop(columns = ['hour','month'],inplace = True)
    return train_data

merged_a = add_cyclic(merged_a)
merged_b = add_cyclic(merged_b)
merged_c = add_cyclic(merged_c)

x_test_a = add_cyclic(x_test_a)
x_test_b = add_cyclic(x_test_b)
x_test_c = add_cyclic(x_test_c)

### Remove outliers during night

In [ ]:
def plot_hourly_avg(y_train):
    # Grouping by hour and calculating the average PV measurement for each hour
    train_data = y_train.copy()
    train_data['hour'] = y_train['time'].dt.hour
    hourly_avg = train_data.groupby('hour')['pv_measurement'].mean()

    # Plotting the average PV production for each hour
    plt.figure(figsize=(12, 6))
    hourly_avg.plot(kind='bar', color='skyblue')
    plt.title('Average PV Production by Hour')
    plt.xlabel('Hour of the Day')
    plt.ylabel('Average PV Production')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

def plot_dist_hour(y_train, hour):
    train_data = y_train.copy()
    train_data['hour'] = y_train['time'].dt.hour
    
    # Filtering the data for the given hour
    hour_data = train_data[train_data['hour'] == hour]
    
    # Plotting the distribution of PV measurements for 1 am
    plt.figure(figsize=(12, 6))
    plt.hist(hour_data['pv_measurement'], bins=50, color='teal', alpha=0.7)
    plt.title(f'Distribution of PV Measurements at {hour}')
    plt.xlabel('PV Measurement')
    plt.ylabel('Frequency')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()
    print(hour_data['pv_measurement'].value_counts())
#train_c[(train_c['time'].dt.hour == 2) &(train_c['pv_measurement'] == 9.8)]

def get_nighttime_stats(y_train,night_start,night_end):
    train_data = y_train.copy()
    train_data['hour'] = y_train['time'].dt.hour

    # Filtering the data for nighttime hours (8 pm to 4 am)
    nighttime_data = train_data[(train_data['hour'] >= night_start) | (train_data['hour'] <= night_end)]

    # Descriptive statistics for nighttime PV measurements
    nighttime_stats = nighttime_data['pv_measurement'].describe()

    # Plotting the distribution of nighttime PV measurements
    plt.figure(figsize=(12, 6))
    plt.hist(nighttime_data['pv_measurement'], bins=50, color='purple', alpha=0.7)
    plt.axvline(nighttime_stats['75%'], color='red', linestyle='dashed', label='75th Percentile')
    plt.axvline(nighttime_stats['max'], color='green', linestyle='dashed', label='Max Value')
    plt.title('Distribution of Nighttime PV Measurements')
    plt.xlabel('PV Measurement')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

    print(nighttime_stats)
    
def set_nighttime_to_zero(y_train, night_start,night_end, thresh):
    df = y_train.copy()
    df['hour'] = y_train['time'].dt.hour
    mask = (df['hour'] >= 23) | (df['hour'] <= 3) & (df['pv_measurement'] > thresh)
    df.loc[mask, 'pv_measurement'] = 0
    df = df.drop(columns = ['hour'])
    return df

#train_a[(train_a['time'].dt.hour == 2) &(train_a['pv_measurement'] >0)]
#train_a = set_nighttime_to_zero(train_a,23,3,0)
#train_b = set_nighttime_to_zero(train_b,23,3,0)
#train_c = set_nighttime_to_zero(train_c,23,3,0)
#train_a[(train_a['time'].dt.hour == 2) &(train_a['pv_measurement'] >0)]

### Remove rows with high rad values and zero PV 

In [ ]:
def remove_rad_null(merged_df):
    merged_data = merged_df.copy()
    merged_data['clear_sky_rad:W'].fillna(0, inplace=True)
    merged_data['clear_sky_rad:W'].fillna(0, inplace=True)
    merged_data['direct_rad:W'].fillna(0, inplace=True)
    merged_data['direct_rad_1h:J'].fillna(0, inplace=True)
    return merged_data
"""
m_a = remove_rad_null(merged_a)
m_b = remove_rad_null(merged_b)
m_c = remove_rad_null(merged_c)
"""

def get_percentiles_df(merged_df):
    merged_data = merged_df.copy()
    merged_data['clear_sky_rad:W'].fillna(0, inplace=True)
    merged_data['clear_sky_rad:W'].fillna(0, inplace=True)
    merged_data['direct_rad:W'].fillna(0, inplace=True)
    merged_data['direct_rad_1h:J'].fillna(0, inplace=True)

    # Calculate and display percentiles
    percentiles = [50,60,70,80,85,90,95]
    percentile_values_direct_rad= np.percentile(merged_data['direct_rad:W'], percentiles)
    percentile_values_direct_rad_1h = np.percentile(merged_data['direct_rad_1h:J'], percentiles)
    percentile_values_clear_sky_rad = np.percentile(merged_data['clear_sky_rad:W'], percentiles)
    percentile_values_clear_sky_energy = np.percentile(merged_data['clear_sky_energy_1h:J'], percentiles)
    percentile_values_df = pd.DataFrame({
        'Percentile': percentiles,
        'direct_rad:W':percentile_values_direct_rad,
        'direct_rad_1h:J': percentile_values_direct_rad_1h,
        'clear_sky_rad:W': percentile_values_clear_sky_rad,
        'clear_sky_energy_1h:J': percentile_values_clear_sky_energy
        })
    
    return percentile_values_df

def get_anomals(merged_data,feature,percentile): 
    #identify the rows where the "direct_rad:W" column in x_train_a is high
    #but the PV measurement in train_a is zero -> Indicates wrong
    
    percentile_df = get_percentiles_df(merged_data)
    
    # Define a threshold for high solar radiation
    threshold = percentile_df[percentile_df['Percentile']==percentile][feature].values[0],

    # Find rows where 'direct_rad:W' is high but PV measurement is zero
    anomalous_rows = merged_data[(merged_data[feature] > threshold) & (merged_data['pv_measurement'] == 0)]
    
    
    # Display the anomalous rows
    return anomalous_rows
"""
merged_a1 = merged_a.copy().drop(get_anomals(merged_a,'clear_sky_rad:W',90).index)
merged_b1 = merged_b.copy().drop(get_anomals(merged_b,'direct_rad:W',90).index)
merged_c1 = merged_c.copy().drop(get_anomals(merged_c,'direct_rad_1h:J',90).index)
"""

### Add avg pv at this time over the past week or month.

In [ ]:
resampled_df = merged_a.resample('7D', on='date_forecast',).mean()
resampled_df['pv_measurement']

In [ ]:
def calculate_rolling_same_time_average(merged_df, period='7D'):
    
    df = merged_df.copy()
    # Resample the data at the desired frequency
    resampled_df = df.resample(period, on='date_forecast',).mean()
    
    
    # Reindex the resampled data to match the original index, filling missing values by interpolation
    return resampled_df

#merged_a['weekly_avg_pv_hourly'] = calculate_rolling_same_time_average(merged_a, '7D')

# Calculate the rolling average at the same time over the past month
#merged_a['monthly_avg_same_time'] = calculate_rolling_same_time_average(merged_a, '30D')

### Add direct_rad * sun_elevation feature

In [ ]:
#Did not improve kaggle score
def add_rad_x_sun(merged_data):
    df = merged_data.copy()
    df['rad_x_sun_elevation'] = df['direct_rad:W']*df['sun_elevation:d']
    return df
"""
mod_a = add_rad_x_sun(merged_a)
mod_b = add_rad_x_sun(merged_b)
mod_c = add_rad_x_sun(merged_c)

x_test_a_mod = add_rad_x_sun(x_test_a)
x_test_b_mod = add_rad_x_sun(x_test_b)
x_test_c_mod = add_rad_x_sun(x_test_c)
"""

### Categorical Feats

In [73]:
import pandas as pd

def convert_columns_to_cat(merged_data, cat_features):
    df = merged_data.copy()
    for col in cat_features:
        df[col] = df[col].astype(str)
    return df

cat_features=['estimated','dew_or_rime:idx','is_day:idx','is_in_shadow:idx','precip_type_5min:idx','snow_drift:idx']
cat_features1 = ['estimated']
cat_features2= ['estimated', 'is_in_shadow:idx', 'precip_type_5min:idx']

merged_a_cat = convert_columns_to_cat(merged_a,cat_features1)
merged_b_cat = convert_columns_to_cat(merged_b,cat_features1)
merged_c_cat = convert_columns_to_cat(merged_c,cat_features1)

x_test_a_cat = convert_columns_to_cat(x_test_a,cat_features1)
x_test_b_cat = convert_columns_to_cat(x_test_b,cat_features1)
x_test_c_cat = convert_columns_to_cat(x_test_c,cat_features1)

## Build Catboost model 

In [ ]:
def build_catboost(merged_df,own_split = False):    
    if own_split:
        train_data, val_data = split_dataset(merged_df)
        train_data.drop(columns=['date_forecast','time'],inplace = True)
        val_data.drop(columns=['date_forecast','time'],inplace = True)
        
        X_train = train_data.drop(columns=['pv_measurement'])
        X_validation = val_data.drop(columns=['pv_measurement'])
        
        y_train = train_data['pv_measurement']
        y_validation = val_data['pv_measurement']
        
    else:
        merged_df = merged_df.drop(columns=['date_forecast','time'])
            
        X = merged_df.drop(columns=['pv_measurement'])
        y = merged_df['pv_measurement']
    
        X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.8, random_state=42)
  
    catboost_model = CatBoostRegressor(
        cat_features=['estimated'],
        iterations=4000,
        learning_rate=0.1,
        depth=6,
        loss_function='MAE',
        eval_metric='MAE',
        random_seed=42,
        verbose=200
    )
    
    catboost_model.fit(X_train, y_train, eval_set=(X_validation, y_validation), use_best_model=True, early_stopping_rounds=200)
    return catboost_model


In [ ]:
model_a = build_catboost(merged_a_cat,False)
model_b = build_catboost(merged_b_cat,False)
model_c = build_catboost(merged_c_cat,False)

In [ ]:
model_a = build_catboost(merged_a)
model_b = build_catboost(merged_b)
model_c = build_catboost(merged_c)

### Build multiple catboost models

In [76]:
def build_catboost_multiple_seed(merged_df,x_test):
    merged_df = merged_df.drop(columns=['date_forecast', 'time'])
    X = merged_df.drop(columns=['pv_measurement'])
    y = merged_df['pv_measurement']
    
    predictions = []
    models = []
    scores = []
    seeds = range(30)  # Random seeds from 0 to 9
    
    for seed in seeds:
        X_train, X_validation, y_train, y_validation = train_test_split(
            X, y, train_size=0.8, random_state=seed)
        
        catboost_model = CatBoostRegressor(
            cat_features=['estimated'],
            iterations=10000,
            learning_rate=0.1,
            depth=6,
            loss_function='MAE',
            eval_metric='MAE',
            random_seed=seed,
            verbose=False
        )
        
        catboost_model.fit(X_train, y_train, eval_set=(X_validation, y_validation),
                           use_best_model=True, early_stopping_rounds=200)
        
        score = catboost_model.get_best_score()['validation']['MAE']
        scores.append(score)
        # Print the best validation MAE for the current seed
        print(f"Best validation MAE for seed {seed}: {score}")
        
        
        # Predict using the current model
        preds = catboost_model.predict(x_test)
        predictions.append(preds)
        models.append(catboost_model)
    
    # Average the predictions from all models
    averaged_predictions = np.mean(predictions, axis=0)
    average_score = np.mean(scores, axis = 0)
    
    return averaged_predictions,models, average_score



In [77]:
pred_a, models_a, avg_a = build_catboost_multiple_seed(merged_a_cat,x_test_a_cat)
pred_b, models_b, avg_b = build_catboost_multiple_seed(merged_b_cat,x_test_b_cat)
pred_c, models_c, avg_c= build_catboost_multiple_seed(merged_c_cat,x_test_c_cat)

Best validation MAE for seed 0: 171.37944334964084
Best validation MAE for seed 1: 176.08212201103308
Best validation MAE for seed 2: 171.05006893630627
Best validation MAE for seed 3: 166.07339279503202
Best validation MAE for seed 4: 167.988943808119
Best validation MAE for seed 5: 170.8150645617657
Best validation MAE for seed 6: 171.1427035297064
Best validation MAE for seed 7: 173.08250777616368
Best validation MAE for seed 8: 167.35751429136945
Best validation MAE for seed 9: 167.9763808397839
Best validation MAE for seed 10: 172.39094422009907
Best validation MAE for seed 11: 174.70258695632134
Best validation MAE for seed 12: 170.12782753995657
Best validation MAE for seed 13: 173.95599485733425
Best validation MAE for seed 14: 168.68232671480627
Best validation MAE for seed 15: 167.56956660591075
Best validation MAE for seed 16: 167.1253103779937
Best validation MAE for seed 17: 166.70762278748137
Best validation MAE for seed 18: 174.7442509294883
Best validation MAE for seed 

In [78]:
print(avg_a, avg_b, avg_c)

172.26171994784642 24.516764747652854 21.771519761667157


### Predict Lag

In [ ]:
import pandas as pd

def predict_with_lag(model, test_data, initial_lag_value, lag_hours=1, column_name='pv_measurement'):
    """
    Predict using a model that requires a lag feature, updating the test set iteratively.

    Parameters:
    model (model object): The trained model used for prediction.
    test_data (pd.DataFrame): The test dataset without the target column.
    initial_lag_value (float): The last known value of the target variable from the training set.
    lag_hours (int): The number of hours to lag.
    column_name (str): The name of the target column.

    Returns:
    pd.Series: A series of predictions for the test dataset.
    """
    predictions = []
    lag_feature_name = f"{column_name}_lag_{lag_hours}h"
    current_lag_value = initial_lag_value
    
    for index, row in test_data.iterrows():
        # Set the current lag value
        row[lag_feature_name] = current_lag_value
        
        # Make a prediction
        prediction = model.predict(row.to_frame().transpose())[0]
        predictions.append(prediction)
        
        # Update the lag value with the current prediction
        current_lag_value = prediction
    
    return pd.Series(predictions, index=test_data.index)

initial_lag_val_a = merged_a.tail(24).iloc[0,52]
initial_lag_val_b = merged_b.tail(24).iloc[0,52]
initial_lag_val_c = merged_c.tail(24).iloc[0,52]

# Then, use the function to make predictions:
laged_pred_a = np.array(predict_with_lag(model=laged_model_a, test_data=x_test_a_laged, 
                               initial_lag_value=initial_lag_val_a, lag_hours=24))
laged_pred_b = np.array(predict_with_lag(model=laged_model_b, test_data=x_test_b_laged, 
                               initial_lag_value=initial_lag_val_a, lag_hours=24))
laged_pred_c = np.array(predict_with_lag(model=laged_model_c, test_data=x_test_c_laged, 
                               initial_lag_value=initial_lag_val_a, lag_hours=24))

### Predict and Submit model

In [ ]:
pred_a = model_a.predict(x_test_a)
pred_b = model_b.predict(x_test_b)
pred_c = model_c.predict(x_test_c)

In [ ]:
pred_a = model_a.predict(x_test_a_cat)
pred_b = model_b.predict(x_test_b_cat)
pred_c = model_c.predict(x_test_c_cat)

In [79]:
def create_sub(pred_a,pred_b,pred_c):
    submission = pd.read_csv('sample_submission.csv')
    submission['prediction'] = np.concatenate([pred_a,pred_b,pred_c])
    submission.loc[submission['prediction'] < 0, 'prediction'] = 0
    return submission

sub = create_sub(pred_a,pred_b,pred_c)
#sub = create_sub(laged_pred_a,laged_pred_b,laged_pred_c)

In [80]:
sub

,id,prediction
0,0,0.392815
1,1,0.271528
2,2,0.269724
3,3,54.984417
4,4,346.915775
...,...,...
2155,2155,78.973194
2156,2156,47.315443
2157,2157,18.056388
2158,2158,4.216985


In [81]:
sub.to_csv(f'Submissions/30modelCatboost.csv', index=False)

In [ ]:
def save_model(model,model_name,location):
    save_directory = 'Saved_models/'+ location.upper()
    os.makedirs(save_directory, exist_ok=True)

    # Define the path to save the model
    model_file_path = os.path.join(save_directory, f'{model_name}.cbm')

    # Save the model
    model.save_model(model_file_path)

    print(f"Model successfully saved at {model_file_path}")
    
save_model(model_a,'cyclical_catBoost','A')
save_model(model_b,'cyclical_catBoost','B')
save_model(model_c,'cyclical_catBoost','C')

### Model Evaluation

In [ ]:
def get_feat_importance(model):
    feats = {'feature':merged_a.drop(columns =['date_forecast','time','pv_measurement']).columns,
         'importance':model.get_feature_importance()}
    df = pd.DataFrame(feats).sort_values('importance',ascending = False)
    return df

In [ ]:
get_feat_importance(model_a)

In [ ]:
def compare_two_preds(pred1,pred2):

    plt.figure(figsize=(10, 8))

    # Scatter plot
    plt.scatter(pred1['prediction'], pred2['prediction'], alpha=0.5)

    # Line of equality (for reference)
    plt.plot([pred1['prediction'].min(), pred1['prediction'].max()],
             [pred2['prediction'].min(), pred2['prediction'].max()],
             color='red', linestyle='--')

    # Labels and title
    plt.xlabel('Predictions from First Model')
    plt.ylabel('Predictions from New model')
    plt.title('Comparison of Predictions from Two Models')

    # Show plot
    plt.grid(True)
    plt.show()

In [ ]:
compare_two_preds(sub,sub_lag)

In [ ]:
def plot_prediction(preds):
    test = pd.read_csv('test.csv')
    predictions= preds['predict'].as_data_frame()
    predictions['time'] = test['time'].unique()
    fig, ax1 = plt.subplots(figsize=(15, 6))
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Prediction', color='tab:blue')
    ax1.plot(predictions['time'], predictions['predict'], color='tab:blue', label='Solar Power Production')
    ax1.tick_params(axis='y', labelcolor='tab:blue')

    fig.tight_layout()
    plt.title(f'Time Series Plot of prediction')
    plt.show()

### Post-Processing

In [ ]:
df = pd.read_csv('merged_average2.csv')
df.loc[df['prediction'] < 8, 'prediction'] = 0

In [ ]:
df.to_csv(f'Submissions/merged_models3.csv', index=False)

In [ ]:
maks = max([train_a['pv_measurement'].max(),train_b['pv_measurement'].max(),train_c['pv_measurement'].max()])


### Appendix

In [ ]:
  """# Plot the distribution of "direct_rad:W"
    plt.figure(figsize=(12, 6))
    sns.histplot(merged_data['direct_rad:W'], bins=50, kde=True)
    plt.title('Distribution of "direct_rad:W"')
    plt.xlabel('Direct Radiation (W)')
    plt.ylabel('Frequency')
    plt.show()

    plt.figure(figsize=(12, 6))
    sns.histplot(merged_data['clear_sky_rad:W'], bins=50, kde=True)
    plt.title('Distribution of "clear_sky_rad:W"')
    plt.xlabel('Direct Radiation (W)')
    plt.ylabel('Frequency')
    plt.show()

    plt.figure(figsize=(12, 6))
    sns.histplot(merged_data['direct_rad_1h:J'], bins=50, kde=True)
    plt.title('Distribution of "direct_rad_1h:J"')
    plt.xlabel('Radiation 1h(J)')
    plt.ylabel('Frequency')
    plt.show()

    plt.figure(figsize=(12, 6))
    sns.histplot(merged_data['clear_sky_energy_1h:J'], bins=50, kde=True)
    plt.title('Distribution of "clear_sky_energy_1h:J"')
    plt.xlabel('Radiation 1h(J)')
    plt.ylabel('Frequency')
    plt.show()"""

    

In [ ]:
def add_week_feat(df):

    df['date_forecast'] = pd.to_datetime(df['date_forecast'])
    
    # Extract week number
    df['week'] = df['date_forecast'].dt.isocalendar().week

    return df

"""
x_train_a = add_week_feat(x_train_a)
x_train_b = add_week_feat(x_train_b)
x_train_c = add_week_feat(x_train_c)

x_test_a = add_week_feat(x_test_a)
x_test_b = add_week_feat(x_test_b)
x_test_c = add_week_feat(x_test_c)
"""

"""
x_train_a = pd.read_csv('cleaned_data_Henning/A/x_train_a.csv')
x_train_b = pd.read_csv('cleaned_data_Henning/B/x_train_b.csv')
x_train_c = pd.read_csv('cleaned_data_Henning/C/x_train_c.csv')

x_test_a = pd.read_csv('cleaned_data_Henning/A/x_test_a.csv')
x_test_b = pd.read_csv('cleaned_data_Henning/B/x_test_b.csv')
x_test_c = pd.read_csv('cleaned_data_Henning/C/x_test_c.csv')

train_a = pd.read_csv('cleaned_data_Henning/A/train_a.csv')
train_b = pd.read_csv('cleaned_data_Henning/B/train_b.csv')
train_c = pd.read_csv('cleaned_data_Henning/C/train_c.csv')
"""

In [ ]:
def split_dataset(train_data, val_size=0.1, val = False, estimated_column = 'estimated'):
    if val: 
        estimated_one = train_data[train_data[estimated_column] == 1]

        #Split the filtered dataset into two
        half_index = len(estimated_one) // 2
        validation_set = estimated_one[half_index:]

        # Combine the first half of observed_zero with the rest of the data where observed != 0
        training_set = pd.concat([train_data[train_data[estimated_column] == 0], estimated_one[:half_index]])
    else:
        split_index = int(train_data.shape[0] * (1 - val_size))
        training_set = train_data.iloc[:split_index]
        validation_set = train_data.iloc[split_index:]
    return training_set, validation_set

"""
  else:
        training_set, validation_set = split_dataset(merged_df, val_size, True)
        X_train = training_set.drop(columns=['pv_measurement'])
        y_train = training_set['pv_measurement']
        X_validation = validation_set.drop(columns=['pv_measurement'])
        y_validation = validation_set['pv_measurement']
"""

In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
import numpy as np

def build_catboost_models_multiple_seed_select_best(merged_df,x_test, n_seeds, select_k):
    merged_df = merged_df.drop(columns=['date_forecast', 'time'])
    X = merged_df.drop(columns=['pv_measurement'])
    y = merged_df['pv_measurement']
    
    seed_mae_scores = []
    models = []
    seeds = range(n_seeds)  
    
    # Train 20 models and track their MAE scores
    for seed in seeds:
        X_train, X_validation, y_train, y_validation = train_test_split(
            X, y, train_size=0.8, random_state=seed, shuffle=True)
        
        catboost_model = CatBoostRegressor(
            #cat_features=['estimated', 'is_in_shadow:idx', 'precip_type_5min:idx'],
            iterations=1000,
            learning_rate=0.1,
            depth=6,
            loss_function='MAE',
            eval_metric='MAE',
            random_seed=seed,
            verbose=200
        )
        
        catboost_model.fit(X_train, y_train, eval_set=(X_validation, y_validation),
                           use_best_model=True, early_stopping_rounds=200)
        
        # Get the best validation MAE for the current seed
        mae_score = catboost_model.get_best_score()['validation']['MAE']
        seed_mae_scores.append((seed, mae_score, catboost_model))
        
        # Print the best validation MAE for the current seed
        print(f"Best validation MAE for seed {seed}: {mae_score}")
    
    # Sort models based on MAE scores and select the k best models
    seed_mae_scores.sort(key=lambda x: x[1])
    best_models = seed_mae_scores[:select_k]
    
    # Predict using the 10 best models
    predictions = []
    for seed, mae_score, model in best_models:
        preds = model.predict(x_test)
        predictions.append(preds)
        models.append(model)
    
    average_best_mae = np.mean([mae_score for seed, mae_score, model in best_models])
    # Average the predictions from the k best models
    averaged_predictions = np.mean(predictions, axis=0)
    
    return averaged_predictions,models,average_best_mae

def split_dataset(train_data, date_column='month', estimated_column='estimated'):
    """
    Splits the dataset into a training set and a validation set.
    The validation set includes approximately 50% of the estimated data, evenly distributed across months.
    Additionally, it includes about half of the observed data for May, June, and July, if present.
    The training set includes all months, excluding the observed data that is included in the validation set.
    
    :param train_data: The original training dataset as a pandas DataFrame.
    :param date_column: The name of the column that contains the month information.
    :param observed_column: The name of the column that indicates if the data is observed.
    :return: A tuple (training_set, validation_set)
    """
    # Work with a copy to avoid modifying the original DataFrame
    train_data = train_data.copy()
    train_data.sort_values(by='date_forecast', inplace=True)
    
    # Separate observed and estimated data
    estimated_data = train_data[train_data[estimated_column] == '1']
    observed_data = train_data[train_data[estimated_column] == '0']
    # Split the estimated data into training and validation sets
    estimated_train, estimated_val = train_test_split(
        estimated_data, test_size=0.5, random_state=42, stratify=estimated_data[date_column]
    )
    
    # Check if there are any observed data for May, June, and July
    if not observed_data[observed_data[date_column].isin([5, 6, 7])].empty:
        observed_may_june_july = observed_data[observed_data[date_column].isin([5, 6, 7])]
        observed_train_mjj, observed_val_mjj = train_test_split(
            observed_may_june_july, test_size=0.5, random_state=42, stratify=observed_may_june_july[date_column]
        )
    else:
        
        observed_train_mjj = pd.DataFrame()
        observed_val_mjj = pd.DataFrame()
    
    # Combine the estimated and observed May, June, July data for the validation set
    validation_set = pd.concat([estimated_val, observed_val_mjj])
    validation_set.sort_values(by='date_forecast', inplace=True)
    
    # The rest of the observed data (excluding May, June, July) will be added to the training set
    observed_rest = observed_data[~observed_data[date_column].isin([5, 6, 7])]
    
    # Combine all training parts for the final training set
    training_set = pd.concat([estimated_train, observed_train_mjj, observed_rest])
    training_set.sort_values(by='date_forecast', inplace=True)
    
    return training_set, validation_set
